# This skripts uses the Emotion Classfier to enrich the truths dataset

In [ ]:
import pandas as pd
import torch
TEST_FLAG = False
NUM_TRUTHS = 10
OUTPUT_PATH = "../../Data/ProcessedData/enriched_comments_emotions.tsv"
DEVICE = torch.device("cpu")

truths_path = "../../Data/ProcessedData/truths_cleaned.tsv"
df_truths = pd.read_csv(truths_path, sep="\t")
print(df_truths.head())

In [ ]:
truths_path = "../../Data/ProcessedData/truths_cleaned.tsv"
df_truths = pd.read_csv(truths_path, sep="\t")
print(df_truths.head())

In [ ]:
comments= df_truths["text"].tolist()

if TEST_FLAG:
    comments = comments[:NUM_TRUTHS]
print(comments[0])

In [ ]:
from CommentClassifier.EmotionCommentClassifierLLM import EmotionCommentClassifierLLM

classifier = EmotionCommentClassifierLLM(comments,device=DEVICE)
for comment in comments[:NUM_TRUTHS]:
    print(f"comment: {comment},\n classification: {classifier.classify_one(comment)}\n\n" )


In [ ]:
labels, label_logits = classifier.classify(temp_dir="./temp_progress", batch_size=32, steps_to_save=100) 
print(labels[:5])
print(label_logits[:5])